In [ ]:
import arcpy

# Ścieżki do danych
network_dataset = "ścieżka_do_network_dataset"
origins = "ścieżka_do_centroidów_heksów"
destinations = "ścieżka_do_centroidów_sąsiadów"
output_layer = "OD_Matrix_Layer"

# Tworzenie warstwy OD Cost Matrix
arcpy.na.MakeODCostMatrixLayer(network_dataset, output_layer, "TravelTime")

# Dodawanie punktów początkowych i końcowych
arcpy.na.AddLocations(output_layer, "Origins", origins)
arcpy.na.AddLocations(output_layer, "Destinations", destinations)

# Uruchomienie analizy
arcpy.na.Solve(output_layer)

# Eksport wyników
arcpy.management.CopyRows(f"{output_layer}\\Lines", "ścieżka_do_wyników")


In [ ]:
resolution = 8  # Możesz dostosować tę wartość


In [ ]:
import h3

def polygon_to_h3(geometry, resolution):
    # Konwertuj geometrię do formatu GeoJSON
    geojson = geometry.__geo_interface__
    # Generuj indeksy H3 pokrywające geometrię
    h3_indices = h3.polyfill(geojson, resolution, geo_json_conformant=True)
    return h3_indices


In [ ]:
from shapely.geometry import Polygon
import pandas as pd

# Lista do przechowywania wyników
h3_list = []

for idx, row in poligony.iterrows():
    geometry = row['geometry']
    h3_indices = polygon_to_h3(geometry, resolution)
    for h3_idx in h3_indices:
        h3_list.append({'h3_index': h3_idx, 'source_id': idx})


In [ ]:
def h3_index_to_polygon(h3_index):
    # Pobierz współrzędne wierzchołków heksagonu
    boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
    # Utwórz poligon
    return Polygon(boundary)

# Dodaj geometrię do danych
for item in h3_list:
    item['geometry'] = h3_index_to_polygon(item['h3_index'])


In [ ]:
# Tworzenie DataFrame
df_h3 = pd.DataFrame(h3_list)

# Tworzenie GeoDataFrame
gdf_h3 = gpd.GeoDataFrame(df_h3, geometry='geometry', crs='EPSG:4326')
gdf_h3 = gdf_h3.drop_duplicates(subset=['h3_index'])



In [ ]:
gdf_h3.to_file('h3_heksagony.shp')


In [ ]:
import h3
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

# Lista heksów
hex_list = [
    '8928308280fffff',
    '8928308280bffff',
    '89283082807ffff',
    # Dodaj więcej identyfikatorów heksów
]

unique_edges = {}

for h in hex_list:
    neighbors = h3.k_ring(h, 1)
    neighbors.discard(h)
    for neighbor in neighbors:
        if neighbor in hex_list:
            # Uzyskaj niekierowany identyfikator krawędzi
            undirected_edge = h3.get_undirected_edge(h, neighbor)
            if undirected_edge not in unique_edges:
                # Uzyskaj skierowany identyfikator krawędzi
                directed_edge = h3.get_directed_edge(h, neighbor)
                # Uzyskaj geometrię krawędzi
                edge_boundary = h3.h3_unidirectional_edge_boundary(directed_edge)
                line = LineString(edge_boundary)
                # Dodaj krawędź do słownika
                unique_edges[undirected_edge] = {
                    'edge_id': undirected_edge,
                    'geometry': line
                }

# Stworzenie GeoDataFrame
edges_gdf = gpd.GeoDataFrame.from_dict(unique_edges, orient='index')
edges_gdf.reset_index(drop=True, inplace=True)
edges_gdf.crs = 'EPSG:4326'

# Opcjonalnie: wizualizacja
edges_gdf.plot()

# Opcjonalnie: zapis do pliku
edges_gdf.to_file('edges.geojson', driver='GeoJSON')
